In [1]:
import numpy as np
import torch
import json
import os

from libraries.dataset        import standardize_dataset
from libraries.graph          import graph_POSCAR_encoding
from libraries.structure      import compute_diffraction_pattern
from torch_geometric.data     import Data
from pymatgen.core            import Structure
from pymatgen.io.vasp.outputs import Vasprun

import sys
sys.path.append('../../UCL/m3gnet')
import ML_library as MLL

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/claudio/cibran/Work/UPC/GenerativeModels/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


ModuleNotFoundError: No module named 'ase'

In [2]:
# In case database is created from scratch (otherwise, it is not being used)
DB_path = '../MP/Loaded_PT'

# Define folder in which all data will be stored
data_folder = 'data/GM_BiSI'

# Generation of graph database for training

Load the datasets, already standardized if possible.

In [3]:
# Generate the raw dataset from scratch, and standardize it

# Read all materials within the database
materials = os.listdir(DB_path)

dataset = []
labels  = []
for material in materials[:2000]:
    try:
        # Try to read the polymorphs
        polymorphs = os.listdir(f'{DB_path}/{material}')
    except:
        continue
    
    print(material)
    for polymorf in polymorphs:
        # Path to folder containing the POSCAR
        path_to_POSCAR = f'{DB_path}/{material}/{polymorf}/POSCAR'
        
        # Check that the folder is valid
        if os.path.exists(path_to_POSCAR):
            print(f'\t{polymorf}')
            
            try:
                # Load pymatgen structure object
                structure = Structure.from_file(path_to_POSCAR)
                
                nodes, edges, attributes = graph_POSCAR_encoding(structure,
                                                                 encoding_type='sphere-images')
            except:
                print(f'Error: {material} {polymorf} not loaded')
                continue

            # Load ground state energy per atom
            #gs_energy = float(np.loadtxt(f'{path_to_POSCAR}/EPA'))

            # Compute diffraction pattern from given structure
            diffraction_pattern = compute_diffraction_pattern(structure, diffraction='neutron')
            
            # Construct temporal graph structure
            graph = Data(x=nodes,
                         edge_index=edges.t().contiguous(),
                         edge_attr=attributes.ravel(),
                         y=torch.tensor(diffraction_pattern, dtype=torch.float)
                        )

            # Append to dataset and labels
            dataset.append(graph)
            labels.append(f'{material}-{polymorf}')

CeSnRh
	P-62m
LuSi
	Cmcm
	P-6m2
Gd2Ti2O7
	Fd-3m
	C2-m
LiAs3H2O9
	P2_1-c
VAg2HgO4
	I-42d
Y5Be6Fe3(SiO5)6
	P1
CuCN2
	Cmcm
BaSrEuCrO6
	F-43m
Ta2N3
	Pnma
Nd6Al43W4
	P6_3-mcm
CdSO4
	Cmcm
	Pmn2_1
	P3m1
	C2-m
Mg6ZnFe
	Amm2
BaCeEuSbO6
	P-1
	F-43m
	P222
Li2Mn2CoO6
	P-1
	Cmce
Na(CrS2)2
	P-1
	I4_1-amd
	Pmmn
	Fd-3m
	Pca2_1
	Pnma
	P2-m
	C2-m
KCu2BiS3
	P-1
SrMg14Sn
	P-6m2
	Amm2
LiZnInF6
	P321
KCe(PO3)4
	P2_1-c
	P2_1
K2NiAs2
	Cmcm
Nb2SiNi3
	P6_3-mmc
CaCoF4
	C2-c
Ta2Co3Si
	P6_3-mmc
LaEr4Se7
	Cm
NdTiFe11N
	Imm2
YTaP2SO12
	Cc
MgAl3SiBO9
	Pnma
CaZn2(H5O4)2
	P2_1-c
K3CeI6
	Fm-3m
Mn3Ni2P6WO24
	R3
H6PbC2S2O7
	P-1
BaFe4O7
	P6_3mc
	P6_3-m
Nb2CoS4
	P-3m1
	Pnma
W2Br5
	P2_1-c
ZrP
	P6_3-mmc
	Fm-3m
BaClF
	P4-nmm
NaPbO3
	Pm-3m
KAl4(SiO6)2
	C2-c
Y4Cu7O16
	P-1
Li5Mn5Cr2O12
	C2
U11Se20
	P4nc
	P4
Yb2MgTi2O6
	R3
	R-3
MgTiMn3O8
	Cm
	P1
	C2-m
Na6Y4Al4Si3(S3O16)3
	P1
LiNi7O7F
	Pm
Na4Ga4Si19
	P1
BaMgFeF7
	P2_1-c
Sn4Te3Se
	R3m
Sr2Cd2Cu(SO)2
	I4-mmm
Na14Ca4Hf4Si7S5O48
	P1
PrTaTiO6
	Pna2_1
Nd(NiO2)2
	Cmce
Mg6ZnCd
	Amm2
CdHg4C6

dataset = []
labels  = []

### Loaded database ###

data_path = 'Loaded_BiSI/gamma'

# Iterate over materials and relaxations in the dataset
while len(dataset) < 200:
    for material in os.listdir(data_path):
        # Define path to material
        path_to_material = f'{data_path}/{material}'

        # Check if it is a folder
        if not os.path.isdir(path_to_material):
            continue

        print()
        print(material)

        # Get relaxations steps (rel1, rel2...)
        relaxation_steps = os.listdir(path_to_material)

        # Determine all defect states across every folder
        defect_states = []
        for relaxation_step in relaxation_steps:
            path_to_relaxation_step = f'{path_to_material}/{relaxation_step}'
            if os.path.isdir(path_to_relaxation_step):
                for defect_state in os.listdir(path_to_relaxation_step):
                    if os.path.isdir(f'{path_to_material}/{relaxation_step}/{defect_state}'):
                        defect_states.append(defect_state)

        # Determine unique defect states across every folder
        unique_defect_states = np.unique(defect_states)

        # Run over all defect states
        for defect_state in unique_defect_states:
            print(f'\t{defect_state}')

            # Run over all relaxation steps
            for relaxation_step in relaxation_steps:
                # Define path to relaxation loading every relaxation step
                # of a same defect state in the same data column
                path_to_deformation = f'{path_to_material}/{relaxation_step}/{defect_state}'

                # Avoiding non-directories (such as .DS_Store)
                if not os.path.isdir(path_to_deformation):
                    continue

                # Define name for the defect state folder
                temp_relaxation = f'{material}_{defect_state}'

                # Check if it is a valid relaxation (with a vasprun.xml file)
                # If not, it might be that there are different deformation folders of the defect state
                if MLL.is_relaxation_folder_valid(path_to_deformation):
                    path_to_relaxations = [path_to_deformation]
                else:
                    # Try to extract deformation folders
                    deformation_folders = os.listdir(path_to_deformation)

                    # Run over deformations
                    path_to_relaxations = []
                    for deformation_folder in deformation_folders:
                        path_to_relaxation = f'{path_to_deformation}/{deformation_folder}'
                        if MLL.is_relaxation_folder_valid(path_to_relaxation):
                            path_to_relaxations.append(path_to_relaxation)

                # Gather relaxations from different deformations as different ionic steps
                for path_to_relaxation in path_to_relaxations:
                    # Remove invalid characters from the vasprun.xml file
                    MLL.clean_vasprun(path_to_relaxation)  # Uncomment is it happens to you as well!!

                    if not os.path.exists(f'{path_to_relaxation}/vasprun.xml'):
                        print(f'Check {path_to_relaxation}')

                    # Load data from relaxation
                    try:
                        # Try to load those unfinished relaxations as well
                        vasprun = Vasprun(f'{path_to_relaxation}/vasprun.xml', exception_on_bad_xml=False)
                    except:
                        print('Error: vasprun not correctly loaded.')
                        continue

                    # Run over ionic steps
                    for ionic_step_idx in range(len(vasprun.ionic_steps)):
                        temp_ionic_step = f'{temp_relaxation}_{ionic_step_idx}'
                        # Extract data from each ionic step
                        temp_structure = vasprun.ionic_steps[ionic_step_idx]['structure']
                        temp_energy    = vasprun.ionic_steps[ionic_step_idx]['e_fr_energy']
                        
                        nodes, edges, attributes = graph_POSCAR_encoding(temp_structure,
                                                                         encoding_type='sphere-images')

                        # Construct temporal graph structure
                        graph = Data(x=nodes,
                                     edge_index=edges.t().contiguous(),
                                     edge_attr=attributes.ravel(),
                                     y=torch.tensor([temp_energy], dtype=torch.float)
                                    )

                        # Append to dataset and labels
                        dataset.append(graph)
                        labels.append(f'')

In [4]:
# Standardize dataset
dataset_std, dataset_parameters = standardize_dataset(dataset)

# Save dataset

In [5]:
labels_name                 = f'{data_folder}/labels.pt'
dataset_name_std            = f'{data_folder}/dataset.pt'
dataset_parameters_name_std = f'{data_folder}/standardized_parameters.json'  # Parameters for rescaling the predictions

if not os.path.exists(data_folder):
    os.system(f'mkdir {data_folder}')

torch.save(labels,  labels_name)
torch.save(dataset, dataset_name_std)

# Convert torch tensors to numpy arrays
numpy_dict = {key: value.cpu().numpy().tolist() for key, value in dataset_parameters.items()}

# Dump the dictionary with numpy arrays to a JSON file
with open(dataset_parameters_name_std, 'w') as json_file:
    json.dump(numpy_dict, json_file)